In [33]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
from csv import writer, reader
import decimal
import pandas as pd

In [34]:
SEASONS = list(range(1997, 2024))
months = ['november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']

In [35]:
DATA_DIR = "testing_data_collection"
STANDINGS_DIR = os.path.join(DATA_DIR, "games")
curr_dir = '/Users/chrisjeff/Desktop/Origin/'
#SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [36]:
#get html function
async def get_html(url, selector, sleep=5, retries=5):
  html = None
  for i in range(1, retries+1):
    time.sleep(sleep * i)

    try:
      async with async_playwright() as p:
        browser = await p.firefox.launch()
        page = await browser.new_page()
        await page.goto(url)
        print(url)
        html = await page.inner_html(selector)
    except PlaywrightTimeout:
      print(f"Timeout on {url}")
      if i == retries:
        print('No games this month')
      continue
    else:
      break
  return html

Gets links for every game for between a range of months, between a range of years

In [11]:
for season in SEASONS:
    for month in months:
        file_of_links = f'games_from_{month}-{season}.csv'
        url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games-{month}.html'
        html = await get_html(url, '#schedule')
        if not html:
            continue

        soup = BeautifulSoup(html, 'html.parser')
        row = soup.find_all('a')
        links = [_['href'] for _ in row]
        all_links = [f"https://basketball-reference.com{game}" for game in links]

        indiv_game_boxscores = []
        for url in all_links:
            if 'boxscores' in url and url[-1] == 'l':
                indiv_game_boxscores.append(url)
        print(indiv_game_boxscores)
        with open(f'{curr_dir}Collecting_Data/Links_to_games/{file_of_links}', 'w') as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(indiv_game_boxscores)
            f_object.close()
            

https://www.basketball-reference.com/leagues/NBA_2023_games-november.html
['https://basketball-reference.com/boxscores/202211010BRK.html', 'https://basketball-reference.com/boxscores/202211010MIA.html', 'https://basketball-reference.com/boxscores/202211010OKC.html', 'https://basketball-reference.com/boxscores/202211010PHO.html', 'https://basketball-reference.com/boxscores/202211020PHI.html', 'https://basketball-reference.com/boxscores/202211020CLE.html', 'https://basketball-reference.com/boxscores/202211020MIA.html', 'https://basketball-reference.com/boxscores/202211020NYK.html', 'https://basketball-reference.com/boxscores/202211020CHI.html', 'https://basketball-reference.com/boxscores/202211020HOU.html', 'https://basketball-reference.com/boxscores/202211020MIL.html', 'https://basketball-reference.com/boxscores/202211020SAS.html', 'https://basketball-reference.com/boxscores/202211020DAL.html', 'https://basketball-reference.com/boxscores/202211020POR.html', 'https://basketball-reference

In [45]:
async def get_game_html_data(url):
    html = await get_html(url, 'sortable stats_table')
    soup = BeautifulSoup(html, 'html.parser')
    return soup
    

In [47]:
for file_number, file_name in enumerate(os.listdir(f'{curr_dir}Collecting_Data/Links_to_games')):
  if os.path.isfile(f'{curr_dir}Collecting_Data/game_html/{new_file}'):
    os.remove(f'{curr_dir}Collecting/game_html/{new_file}')
  print(file_name)
  new_file = f'game_{file_number+1}'

  with open(f'{curr_dir}Collecting_Data/Links_to_games/{file_name}', 'r') as file_object:
    temp = reader(file_object)

    for row in temp:
      #print(links)
      for link in row:
        with open(f'{curr_dir}Collecting_Data/game_html/game_{file_number+1}', 'w'):
          await get_game_html_data(link)
          

games_from_march-2004.csv
https://basketball-reference.com/boxscores/200403010BOS.html
Timeout on https://basketball-reference.com/boxscores/200403010BOS.html
https://basketball-reference.com/boxscores/200403010BOS.html
Timeout on https://basketball-reference.com/boxscores/200403010BOS.html
https://basketball-reference.com/boxscores/200403010BOS.html
Timeout on https://basketball-reference.com/boxscores/200403010BOS.html
https://basketball-reference.com/boxscores/200403010BOS.html
Timeout on https://basketball-reference.com/boxscores/200403010BOS.html
https://basketball-reference.com/boxscores/200403010BOS.html
Timeout on https://basketball-reference.com/boxscores/200403010BOS.html
No games this month


TypeError: object of type 'NoneType' has no len()

In [52]:
def min_to_sec(time):        #converts time to an integer
    if time == 'Did Not Play' or time == 'Did Not Dress' or time == None or time == '\xa0':
        return 0
    
    time_list = time.split(':')
    return int(time_list[0]) * 60 + int(time_list[1])

def int_conv(num):           #converts number to int
    if num == '':
        return None
    
    return int(num)

def float_conv(num):       #converts number to float to nearest 1000th place, rounding down
    if num == '':
        return None
    
    return float(Decimal(str(num)).quantize(Decimal('.001'), rounding=decimal.ROUND_DOWN))

def plus_minus_conv(val):
    if val == '':
        return None
    if val == '0':
        return 0
    
    value_symbol = val[0]
    num = int(val[1:])
    if value_symbol == '-':
        return -num
    else:
        return num


converters = [str, min_to_sec, int_conv, int_conv, float_conv, int_conv, int_conv, float_conv,
              int_conv, int_conv, float_conv, int_conv, int_conv, int_conv, int_conv, int_conv,
              int_conv, int_conv, int_conv, int_conv, plus_minus_conv]


directory = 'Temp_Data'
useful_away_tables = {3: 'Q1', 4: 'Q2', 5: 'H1', 6: 'Q3', 7: 'Q4', 8: 'H2'}
useful_home_tables = {11: 'Q1', 12: 'Q2', 13: 'H1', 14: 'Q3', 15: 'Q4', 16: 'H2'}
headers = ['Starters', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
           'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-']

Print all players stats for a given quarter

In [ ]:
for filename in os.listdir(directory):
    print(filename)

    soup = BeautifulSoup(open(f'/Users/chrisjeff/Desktop/Senior-Project/Senior_project/Temp_Data/{filename}'), 'html.parser')
    tables = soup.find_all('table', class_='stats_table')

    player_data = []
    amount_of_players = 0
    
    for _, q in enumerate(useful_away_tables):
        #for head in tables[q].find('thead').find_all('tr', attrs={'class': None}):
        #    headers.append(head.find('th').text)
        for row in tables[q].find('tbody').find_all('tr', attrs={'class': None}):
            player_name = row.find('th').text
            #names.append(player_name)
            data_row = ','.join(e.text for e in row.children)
            player_data.append(data_row)
                
    quarter_index = 0
    amount_of_players = len(player_data) // 6

    with open('cleaned_data.csv', 'a') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(headers)
        f_object.close()
        
    for n in player_data:
        if quarter_index % amount_of_players == 0:
            print('end of term')
        quarter_index += 1

        row = n.split(',')
        results = []
        if len(row) > 2:
            for i, func in enumerate(converters):
                converted_val = func(row[i])
                results.append(converted_val)
        else:
            row[1] = None
            results.append(row[0])
            results.append(row[1])

        print(results)
        with open('cleaned_data.csv', 'a') as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(results)
            f_object.close()


#0, 2, 5, 9, 14, 20   :   Indices of tables that matter
#Get player info for each table

Test writing to .csv

In [54]:
for file_number, filename in enumerate(os.listdir(directory)):
    new_file = 'cleaned_data_game_{}.csv'.format(file_number+1)
    curr_dir = '/Users/chrisjeff/Desktop/Origin/'

    if os.path.isfile(f'{curr_dir}Cleaned_data/{new_file}'):
        os.remove(f'{curr_dir}Cleaned_data/{new_file}')
    print(filename)

    soup = BeautifulSoup(open(f'{curr_dir}Temp_Data/{filename}'), 'html.parser')
    tables = soup.find_all('table', class_='sortable stats_table')
    num_of_tables = 6
    curr_team = useful_home_tables

    amount_of_players = len(tables[0].find('tbody').find_all('tr', attrs={'class': None}))
    player_data = []
    
    for team_num in range(2):
        for player_number in range(amount_of_players):
            for index in range(len(curr_team)):
                name = tables[index + (team_num*6)].find('tbody').find_all('tr', attrs={'class': None})[player_number]
                #player_name = row.find('th').text
                #names.append(player_name)
                #print(name)
                data_row = ','.join(e.text for e in name.children)
                player_data.append(data_row)


    #print(player_data)

    
    with open(f'{curr_dir}Cleaned_data/{new_file}', 'w') as f_object: #Write headers to every file needed
        writer_object = writer(f_object)
        writer_object.writerow(headers)
        f_object.close()

    
    quarter_index = 0

    for i, n in enumerate(player_data):  #separates stats for each player from every term
        if (quarter_index) % ((len(tables) // 2)) == 0:
            print('end of player stats')
        quarter_index += 1
    
        results = []
        row = n.split(',')
        if len(row) > 2:
            for i, func in enumerate(converters): #Have player_data get converted from str to appropriate data type
                converted_val = func(row[i])
                results.append(converted_val)
        else:
            row[1] = None #Data cell is empty (hasn't played)
            results.append(row[0])
            results.append(row[1])

        print(results)
        with open(f'{curr_dir}Cleaned_data/{new_file}', 'a') as f_object: #writes player data to new file
            writer_object = writer(f_object)
            writer_object.writerow(results)
            f_object.close()
    


#0, 2, 5, 9, 14, 20   :   Indices of tables that matter
#Get player info for each tabl

201510270ATL.html


end of player stats
['Andre Drummond', 599, 2, 5, 0.4, 0, 0, None, 0, 2, 0.0, 0, 3, 3, 1, 1, 0, 0, 0, 4, 7]
['Andre Drummond', 489, 1, 2, 0.5, 0, 0, None, 0, 0, None, 1, 2, 3, 1, 0, 1, 1, 1, 2, 7]
['Andre Drummond', 1088, 3, 7, 0.429, 0, 0, None, 0, 2, 0.0, 1, 5, 6, 2, 1, 1, 1, 1, 6, 14]
['Andre Drummond', 720, 3, 7, 0.429, 0, 0, None, 5, 6, 0.833, 3, 3, 6, 1, 0, 0, 1, 1, 11, 11]
['Andre Drummond', 421, 0, 2, 0.0, 0, 0, None, 1, 2, 0.5, 4, 3, 7, 0, 0, 1, 0, 2, 1, -2]
['Andre Drummond', 1141, 3, 9, 0.333, 0, 0, None, 6, 8, 0.75, 7, 6, 13, 1, 0, 1, 1, 3, 12, 9]
end of player stats
['Marcus Morris', 720, 3, 7, 0.429, 0, 1, 0.0, 1, 2, 0.5, 1, 2, 3, 2, 0, 0, 0, 0, 7, 0]
['Marcus Morris', 347, 1, 2, 0.5, 1, 1, 1.0, 0, 0, None, 1, 2, 3, 0, 0, 0, 0, 0, 3, 10]
['Marcus Morris', 1067, 4, 9, 0.444, 1, 2, 0.5, 1, 2, 0.5, 2, 4, 6, 2, 0, 0, 0, 0, 10, 10]
['Marcus Morris', 720, 1, 5, 0.2, 0, 2, 0.0, 0, 0, None, 1, 1, 2, 2, 0, 0, 0, 1, 2, 11]
['Marcus Morris', 438, 1, 5, 0.2, 0, 0, None, 4, 4, 1.0, 2,